<a href="https://colab.research.google.com/github/phillipSloan/ComputationalLogic/blob/prolexa-plus/Report%20Notebook%20Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report for COMSM0022 Computational Logic for Artificial Intelligence (2021)

## created by [Phillip Sloan](https://gibhub.com/phillipSloan) and [Jonathan Erskine](https://github.com/jmerskine1)

### Introduction

This report demonstrates our approach to this assignment, walking through how we implemented negation and other command line code. It seeks to explain our thought process, which was wrong at certain points, causing unwanted proofs from the meta-interpreter. There are several snippets of code throughout this report of how our code was created, with the Colab notebook demonstrating the final implementation. A link to it can be found at the top of this report.


## First we get Prolexa running:
### Install SWI-Prolog & Prolexa

In [1]:
!apt-get install swi-prolog -qqq > /dev/null
!yes | pip install git+https://github.com/phillipSloan/ComputationalLogic/ -qqq > /dev/null

from pyswip import Prolog
import prolexa.meta_grammar as meta

# Added this due to an error with meta
import nltk
nltk.download('omw-1.4')
  

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

# clearing the output to keep it tidy
from IPython.display import clear_output 
clear_output()

### Add a standard few rules
First check the traditional one work:

In [5]:
print(meta.standardised_query(pl, "spill the beans")[0]['Output'])

print(meta.standardised_query(pl, "peter is human")[0]['Output'])
print(meta.standardised_query(pl, "every human is mortal")[0]['Output'])

print(meta.standardised_query(pl, "explain why peter is mortal")[0]['Output'])

donald is not happy. donald is not happy
I will remember that peter is human
I will remember that every human is mortal
peter is human; every human is mortal; therefore peter is mortal


##Understanding the meta-interpreter

We decided to focus on adding negation to Prolexa. Which asked us to implement: 

>Every teacher is happy. Donald is not happy. Therefore, Donald is not a teacher.

Our first thoughts were to add the rule "Every teacher is happy" and "Donald is happy" to the stored rules, and see if Prolexa could understand this logic.

In [6]:
print(meta.standardised_query(pl, "every teacher is happy")[0]['Output'])
print(meta.standardised_query(pl, "donald is happy")[0]['Output'])
print(meta.standardised_query(pl, "explain why donald is a teacher")[0]['Output'])

I will remember that every teacher is happy
I will remember that donald is happy
Sorry, I don't think this is the case


We thought it was strange that the meta-interpreter could not handle the second case. Looking at the existing prove_rb meta interpreter:

```prolog
prove_rb(true,_Rulebase,P,P):-!.
prove_rb((A,B),Rulebase,P0,P):-!,
	find_clause((A:-C),Rule,Rulebase),
	conj_append(C,B,D),
    prove_rb(D,Rulebase,[p((A,B),Rule)|P0],P).
prove_rb(A,Rulebase,P0,P):-
    find_clause((A:-B),Rule,Rulebase),
	prove_rb(B,Rulebase,[p(A,Rule)|P0],P).
```

When looking at the final predicate, it can be seen that prove_rb/4 tries to unify B with a body of a stored rule which head matches the given A. It will cycle through all stored_rules and try and find a matching rule, but in the second case this will not happen, as it will be trying to match teacher(donald) to happy(X), so it fails. We realised that a new predicate needed to be created, that mirrored the existing one and unified the head A, given a body B. The following predicate was added to prove_rb/4.

```prolog
prove_rb(B,Rulebase,P0,P):-
  find_clause((A:-B),Rule,Rulebase),
	prove_rb(A,Rulebase,[p(B,Rule)|P0],P).
```
Adding this statement allowed Prolexa to prove the second statement. However, when discussing this meta-interpreter we realised there was a flaw in our intial logic. We should not be trying to prove `teacher(donald)` from `happy(X):-teacher(X).` as it doesn't follow logically from it.  The rule means `Every teacher is happy`. Teachers are a subset of happy, but the rule doesn't provide the information that all happy X's are teachers. 

Instead of `happy(X):-teacher(X)`, we need to have the rule `teacher(X):-happy(X)` which logically means if happy then teacher.

This meant we had to add the following to prolexa_grammar.pl:

```prolog
sentence1(C) --> flipped_verb_phrase(N,M1), flipped_determiner(N,M1,M2,C),  noun(N,M2).

flipped_verb_phrase(s,M) --> [if],property(s,M).
flipped_verb_phrase(p,M) --> [if],property(p,M).

flipped_determiner(s,X=>B,X=>H,[(H:-B)]) --> [then].
flipped_determiner(p,X=>B,X=>H,[(H:-B)]) --> [then].
```

### Negation
To implement negation we suplemented the existing prolexa .pl files, prolexa_grammar.pl was extended with the following code:


```prolog
:-op(900,fy,not).

sentence1([(H:-not(B))]) --> determiner(N,M1,M2,[(H:-B)]),noun(N,M1),verb_phrase(N,not(M2)).
sentence1([(not(L):-true)]) --> proper_noun(N,X),verb_phrase(N,not(X=>L)).

verb_phrase(s,not(M)) --> [is],[not],property(s,M).
verb_phrase(s,not(M)) --> [not],property(s,M).

question1(not(Q)) --> [who],verb_phrase(s,not(_X=>Q)).
```
The op:- not code was used from the simply logical book / lectures. sentence1 was extended to allow negation of rules such as `not(happy(donald)):-true.` and `teacher(X):-not(happy(X)).`. Although the first is not needed for the assignments task, it was added for completeness and for adding our own rules. verb_phrase was also extended to allow the grammar to cope with `<noun> is not <verb>` as input. question1's addition allows `who is not <verb>` to be utilized.







In [7]:
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])

b'I am a blank slate'
I will remember that donald is not happy


The following code was added to prolexa_engine.pl, which was derived from the existing prove_rb clause.:

```prolog
prove_rb(not A,Rulebase,P0,P):-
  find_clause((A:-B),Rule,Rulebase),
	prove_rb(not B,Rulebase,[p( not A,Rule)|P0],P).
```




 We now add a rule stating that Donald isn't happy which shows our implementation of negation implementing the stored rule `not(happy(donald)):-true.`.

It also demonstrates the remove_conflicting_rules predicate working, as the previous happy rule rule is removed.



Adding `happy(X):-teacher(X)` and then using "explain" to demonstrate the working negated prove_rb meta-interpreter.

In [9]:
print(meta.standardised_query(pl, "if happy then teacher")[0]['Output'])
print(meta.standardised_query(pl, "explain why donald is not a teacher")[0]['Output'])

I will remember that if happy then teacher
donald is not happy; if happy then teacher; therefore donald is not teacher


Final two demonstrate the negation working in Prolexa with the questions "Who and "Is" 

In [10]:
print(meta.standardised_query(pl, "who is not a teacher")[0]['Output'])
print(meta.standardised_query(pl, "is donald not a teacher")[0]['Output'])

b'donald is not teacher'
b'donald is not teacher'


### For further testing

In [ ]:
input = ''  #@param {type:"string"}
print(input)
first_answer = meta.standardised_query(pl, input)[0]['Output']
print(first_answer)

tell me everything you know
peter is human. every human is mortal. donald is not happy. donald is not happy. every teacher is happy
